# 人工知能とソフトコンピューティング 第11回 データ取得 演習 (前半)
## GitHub からのデータ取得準備

### ステップ1
必要なライブラリをインポートする．

In [ ]:
import os # コマンド実行
import pandas as pd # データ分析用ライブラリ
from collections import Counter # カウンタ計測用ライブラリ

### ステップ2
設定読み出し用のライブラリ，GitHub を解析するためのライブラリをインポートする
* util.py: 設定ファイルから読みだした変数を保持するライブラリ
* config.py: 設定ファイルを読み出すライブラリ（util.py から利用）
* config.json: 設定ファイル
* git.py: GitHub 解析するためのライブラリ
* cmdexec.py: コマンド実行のためのライブラリ（git.py から利用）

In [ ]:
import util
from git import Repository

### ステップ3
コミット解析用の関数定義．与えられたコミットでの追加行数，削除行数，変更行数を取得．引数は次の通り．
* cnt: ループ回数（表示用）
* commit_hash: 解析対象のコミットのID
* parent: 解析対象のコミットの親コミットの ID
* repo: 対象のリポジトリオブジェクト
* all_cnt: すべてのコミット数（表示用）

In [ ]:
def parse_commit(cnt, commit_hash, parent, repo, all_cnt):
    print("Processing " + str(cnt + 1) + " of " + str(all_cnt) + " commits.")
    changed_files = repo.get_changed_files(parent, commit_hash)

    diff_counter = Counter()
    for changed_file in changed_files:
        # Update LA and LD
        la, ld = repo.file_stat(parent, commit_hash, changed_file)
        diff_counter.update(Counter(la=la, ld=ld, changed=la+ld))
    
    if len(diff_counter) == 0: # コミットで変更がなかった時
        diff_counter.update(Counter(la=0, ld=0, changed=0))
        
    return (commit_hash, diff_counter)

### ステップ4
リポジトリ全体でのメトリクス解析用の関数定義．全コミットに対して，parse_commit 関数により追加，削除，変更行数を取得する．引数は次の通り．
* repo: 対象のリポジトリオブジェクト

In [ ]:
def extract_metrics(repo):
    commit_list = list(repo.all_commit_hashes())
    #commit_list = list(repo.all_main_branch_hashes())
    all_cnt = len(commit_list)
    print("Total count of commits: " + str(all_cnt))
    
    data_list = []
    parent = None
    for cnt, (commit_hash, datetime, message) in enumerate(commit_list):
        data_list.append(parse_commit(cnt, commit_hash, parent, repo, all_cnt) + (datetime, message))
        parent = commit_hash

    df = pd.DataFrame()
    for data in data_list:
        commit_hash, diff_counter, datetime, message = data
        df = pd.concat([df, pd.DataFrame({'commit_hash':[commit_hash], 'add':[diff_counter['la']], 'delete':[diff_counter['ld']], 'changed':[diff_counter['changed']], 'timestamp':[datetime], 'message':[message]})])

    return df.reset_index(drop=True)

### ステップ5
リポジトリ URL の定義

In [ ]:
repo_url = 'https://github.com/apache/httpd.git'
#repo_url = 'https://github.com/owncloud/android.git' # 別のプロジェクトの例

### ステップ6
リポジトリをクローンし，メトリクスを取得し，CSV に保存（数時間実行時間を要します）．

In [ ]:
repo = Repository(repo_url)
if not repo.is_cloned():
    repo.clone()
df = extract_metrics(repo)
base = os.path.join(util._project_root(), util.data_dir() , "")
csvf = base + repo.name.replace('/', '_') + '.csv'
df.to_csv(csvf)
print("The commit data is saved in " + csvf)

In [ ]:
df # 取得したデータの確認